In [1]:
import numpy as np
from sympy import symbols, cos, sin, pi, simplify, sqrt, atan2, asin, acos
from sympy.matrices import Matrix

In [2]:
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

In [3]:
def rot_x(q):
    M1 = Matrix([[0],[0],[0]])
    M2 = Matrix([[0, 0, 0, 1]])
    Rx = Matrix([[  1,      0,       0],
                 [  0, cos(q), -sin(q)],
                 [  0, sin(q),  cos(q)]])
    rot_x = Rx.row_join(M1)
    rot_x = rot_x.col_join(M2)
    return rot_x
    
def rot_y(q):   
    M1 = Matrix([[0],[0],[0]])
    M2 = Matrix([[0, 0, 0, 1]])
    Ry = Matrix([[ cos(q),     0, sin(q)],
                 [      0,     1,      0],
                 [-sin(q),     0, cos(q)]])
    rot_y = Ry.row_join(M1)
    rot_y = rot_y.col_join(M2)
    return rot_y

def rot_z(q):   
    M1 = Matrix([[0],[0],[0]])
    M2 = Matrix([[0, 0, 0, 1]])
    Rz = Matrix([[ cos(q),-sin(q), 0],
                 [ sin(q), cos(q), 0],
                 [      0,      0, 1]])
    rot_z = Rz.row_join(M1)
    rot_z = rot_z.col_join(M2)
    return rot_z

def trans_x(d):
    M1 = Matrix(np.eye(3))
    M2 = Matrix([[d],[0],[0]])
    M3 = Matrix([[0, 0, 0, 1]])
    Trans_x = M1.row_join(M2)
    Trans_x = Trans_x.col_join(M3)
    return Trans_x

def trans_y(d):
    M1 = Matrix(np.eye(3))
    M2 = Matrix([[0],[d],[0]])
    M3 = Matrix([[0, 0, 0, 1]])
    Trans_y = M1.row_join(M2)
    Trans_y = Trans_y.col_join(M3)
    return Trans_y

def trans_z(d):
    M1 = Matrix(np.eye(3))
    M2 = Matrix([[0],[0],[d]])
    M3 = Matrix([[0, 0, 0, 1]])
    Trans_z = M1.row_join(M2)
    Trans_z = Trans_z.col_join(M3)
    return Trans_z

def DH_transform(q1, d1, alpha0, a0):
    '''The modified DH convention transform matrix
    alpha0: twist angle, a0: link length, 
    d1: link offset, q1: joint angle'''
    T0_1 = Matrix([[cos(q1), -sin(q1),      0,    a0],
                   [sin(q1) * cos(alpha0), cos(q1) * cos(alpha0), -sin(alpha0), -sin(alpha0) * d1],
                   [sin(q1) * sin(alpha0), cos(q1) * sin(alpha0),  cos(alpha0),  cos(alpha0) * d1],
                   [0,              0,      0,     1]])
    return T0_1

In [4]:
# The Modified DH params
s = {alpha0:       0, a0:      0, d1:  0.75,
     alpha1: -pi / 2, a1:   0.35, d2:     0, q2: q2 - pi / 2,
     alpha2:       0, a2:   1.25, d3:     0,
     alpha3: -pi / 2, a3: -0.054, d4:   1.5,
     alpha4:  pi / 2, a4:      0, d5:     0,
     alpha5: -pi / 2, a5:      0, d6:     0,
     alpha6:       0, a6:      0, d7: 0.303, q7: 0}

In [5]:
# Define Modified DH Transformation matrix
# base_link to link1
T0_1 = DH_transform(q1, d1, alpha0, a0)
T0_1 = T0_1.subs(s)
# linke1 to link 2
T1_2 = DH_transform(q2, d2, alpha1, a1)
T1_2 = T1_2.subs(s)
# link2 to link3
T2_3 = DH_transform(q3, d3, alpha2, a2)
T2_3 = T2_3.subs(s)
# link3 to link4
T3_4 = DH_transform(q4, d4, alpha3, a3)
T3_4 = T3_4.subs(s)
# link4 to link5
T4_5 = DH_transform(q5, d5, alpha4, a4)
T4_5 = T4_5.subs(s)
# link5 to link6
T5_6 = DH_transform(q6, d6, alpha5, a5)
T5_6 = T5_6.subs(s)
# link6 to end-effector
T6_7 = DH_transform(q7, d7, alpha6, a6)
T6_7 = T6_7.subs(s)

In [6]:
# Create individual transformation matrices
T0_2 = simplify(T0_1 * T1_2)
T0_3 = simplify(T0_2 * T2_3)
T0_4 = simplify(T0_3 * T3_4)
T0_5 = simplify(T0_4 * T4_5)
T0_6 = simplify(T0_5 * T5_6)
T0_G = simplify(T0_6 * T6_7)

R_corr = rot_z(pi) * rot_y(-pi/2)
T_total = simplify(T0_G * R_corr)

In [ ]:
import pickle
import os
if not os.path.exists("T_final.p"):
    pickle.dump(T0_1, open("T0_1.p", "wb"))
    pickle.dump(T0_2, open("T0_2.p", "wb"))
    pickle.dump(T0_3, open("T0_3.p", "wb"))
    pickle.dump(T0_4, open("T0_4.p", "wb"))
    pickle.dump(T0_5, open("T0_5.p", "wb"))
    pickle.dump(T0_6, open("T0_6.p", "wb"))
    pickle.dump(T0_G, open("T0_G.p", "wb"))
    pickle.dump(T_total, open("T_final.p", "wb"))
else:
    T0_1 = pickle.load(open( "T0_1.p", "rb" ) )
    T0_2 = pickle.load(open( "T0_2.p", "rb" ) )
    T0_3 = pickle.load(open( "T0_3.p", "rb" ) )
    T0_4 = pickle.load(open( "T0_4.p", "rb" ) )
    T0_5 = pickle.load(open( "T0_5.p", "rb" ) )
    T0_6 = pickle.load(open( "T0_6.p", "rb" ) )
    T0_G = pickle.load(open( "T0_G.p", "rb" ) )
    T_total = pickle.load(open( "T_total.p", "rb" ) )

In [77]:
# # Numerically evaluate transforms
state = {q1:	-2.38, q2:	0.38, q3: 0.22, 
         q4: 	4.13, q5:	-0.33, q6:	-4.16}
# # print("T0_1 = ", T0_1.evalf(subs=state))
# # print("T0_2 = ", T0_2.evalf(subs=state))
# # print("T0_3 = ", T0_3.evalf(subs=state))
# # print("T0_4 = ", T0_4.evalf(subs=state))
# print("T0_5 = ", T0_5.evalf(subs=state))
# print("T0_6 = ", T0_6.evalf(subs=state))
# print("T0_G = ", T0_G.evalf(subs=state))
print("T_total = ", T_total.evalf(subs=state))

T_total =  Matrix([
[-0.305510447647209,  0.829998323768836, -0.466654207009134, -1.55536211615731],
[-0.665227018438015, -0.536697343476575,  -0.51906548281046, -1.59631802834278],
[-0.681275553882914,  0.151851058758604,   0.71610395588589, 0.812872468346439],
[                 0,                  0,                  0,               1.0]])


In [78]:
# gripper position from FK calculation
position_fk = T_total.evalf(subs=state)[0:3,3]
# gripper position from rviz simulator
position_g = Matrix([[-1.5438], [-1.601], [0.81447]])

In [79]:
position_error = position_fk - position_g
print("Calculation error: ", position_error)

Calculation error:  Matrix([
[ -0.0115621161573147],
[ 0.00468197165721529],
[-0.00159753165356069]])


In [80]:
position_error.norm()

0.0125759888684085

### Inverse Kinematics

In [44]:
def rot_roll(q):
    R_x = Matrix([[      1,      0,      0],
                  [      0, cos(q), -sin(q)],
                  [      0, sin(q),  cos(q)]])
    return R_x
    
def rot_pitch(q):              
    R_y = Matrix([[ cos(q),     0, sin(q)],
                  [      0,     1,      0],
                  [-sin(q),     0, cos(q)]]) 
    return R_y

def rot_yaw(q):    
    R_z = Matrix([[ cos(q),-sin(q), 0],
                  [ sin(q), cos(q), 0],
                  [      0,      0, 1]])  
    return R_z

In [69]:
simplify(rot_roll(r) * rot_pitch(p) * rot_yaw(y))

Matrix([
[                        cos(p)*cos(y),                        -sin(y)*cos(p),         sin(p)],
[ sin(p)*sin(r)*cos(y) + sin(y)*cos(r), -sin(p)*sin(r)*sin(y) + cos(r)*cos(y), -sin(r)*cos(p)],
[-sin(p)*cos(r)*cos(y) + sin(r)*sin(y),  sin(p)*sin(y)*cos(r) + sin(r)*cos(y),  cos(p)*cos(r)]])

In [70]:
simplify(rot_yaw(y)* rot_pitch(p) * rot_roll(r))

Matrix([
[cos(p)*cos(y), sin(p)*sin(r)*cos(y) - sin(y)*cos(r), sin(p)*cos(r)*cos(y) + sin(r)*sin(y)],
[sin(y)*cos(p), sin(p)*sin(r)*sin(y) + cos(r)*cos(y), sin(p)*sin(y)*cos(r) - sin(r)*cos(y)],
[      -sin(p),                        sin(r)*cos(p),                        cos(p)*cos(r)]])

In [62]:
r, p, y = symbols("r p y") # end-effector orientation
# R0_g_symp = simplify(rot_roll(r) * rot_pitch(p) * rot_yaw(y))
R0_g_symp = simplify( rot_yaw(y)* rot_pitch(p) * rot_roll(r))
R_corr = rot_yaw(pi) * rot_pitch(-pi/2)

In [63]:
# px, py, pz = 2.1529, 0, 1.9465 # initial position
px, py, pz = -0.43776, 1.7479, 1.5323
roll, pitch, yaw = 0.484, 0.015, 0.146
# roll, pitch, yaw =0,0,0
pg_0 = Matrix([[px],[py],[pz]])
R0_g = R0_g_symp.evalf(subs={r: roll, p: pitch, y: yaw})

In [48]:
Q_To_E(0.23936, -0.010299, 0.072359, 0.96818)

(0.481022640223353, -0.0546094106588307, 0.135801571378406)

### Wrist center and theta1

In [68]:
pwc_0 = simplify(pg_0 - (0.303) * R0_g[0:3, 0:3]  * R_corr * Matrix([[0],[0],[1]]))
pwc_0

Matrix([
[-0.462251939833829],
[   1.8868077233368],
[  1.26413253500739]])

In [65]:
p_link5 = Matrix([[-0.73753],[1.7069],[1.158]])

In [66]:
p5_error = pwc_0 - p_link5
p5_error

Matrix([
[ 2.73658376828134e-5],
[-0.00307604553116825],
[   0.378844829564418]])

In [67]:
theta1 = atan2(pwc_0[1], pwc_0[0])
theta1

-1.16229428808577 + pi

### theta3 and theta2

In [ ]:
p2_sym = T0_2 * Matrix([0,0,0,1])
p2_0 = p2_sym.evalf(subs={q1: theta1})
pwc_2 = pwc_0 - p2_0[0:3,:]

In [ ]:
l23 = a2
l35 = sqrt(a3**2 + d4**2)
p25 = sqrt(np.sum(np.square(pwc_2)))

In [ ]:
theta3_1 = atan2(a3,d4)
c235 = (np.sum(np.square(pwc_2)) - l23**2 - l35**2) / (2*l23*l35)
theta3_phi = atan2(sqrt(1-c235**2), c235)
# theta3_phi = atan2(-sqrt(1-c235**2), c235)
theta3 = (theta3_phi + theta3_1 - pi/2).subs(s)
theta3

In [ ]:
theta2_out = atan2(pwc_2[2], sqrt(pwc_2[0]**2 + pwc_2[1]**2))
c523 = (-l35**2 + l23**2 + p25**2) / (2*l23 * p25)
theta2_phi = atan2(sqrt(1 - c523**2), c523)
theta2 = (pi/2 - (theta2_phi + theta2_out)).subs(s)
theta2

### theta5, 4, 6

In [ ]:
R0_3 = T0_3[0:3, 0:3]
R0_3_inv = simplify(R0_3 ** -1)

In [ ]:
R0_3_inv_value = R0_3_inv.evalf(subs={q1:theta1, q2:theta2, q3:theta3})
R3_6 =  R0_3_inv_value * R0_g
R3_6

In [ ]:
R3_6 = Matrix([[.5, -0.1464, 0.8536],[.5,.8536,-0.1464],[-0.7071,.5,.5]])

In [ ]:
r11, r21, r31 = R3_6[0,0], R3_6[1,0], R3_6[2,0]
r32, r33 = R3_6[2,1], R3_6[2,2]
r12, r13 = R3_6[0,1], R3_6[0,2]
r23 = R3_6[1,2]
# beta:q5, alpha:q6, gamma: q4.
# Euler angles from rotation matrix
theta5 = atan2(-r31, sqrt(r11**2 + r21**2)) # y ,theta
theta4 = atan2(r32, r33)# x ,psi
theta6 = atan2(r21, r11) # z ,phi

In [ ]:
theta4, theta5, theta6

In [ ]:
# alpha, beta, gamma
# phi, theta, psi

In [ ]:
x = -6.5
np.abs(x) % (2) * np.sign(x)

In [ ]:
rot_z(pi) * rot_y(-pi/2)

In [ ]:
if np.abs(r31) is not 1:
    beta = atan2(-r31, sqrt(r11**2 + r21**2))
    gamma = atan2(r32/cos(beta), r33/cos(beta))
    alpha = atan2(r21/cos(beta), r11/cos(beta))
else:
    alpha = 0
    if r31 == -1:
        beta = pi/2
        gamma = alpha + atan2(r12, r13)
    else:
        beta = -pi/2
        gamma = -alpha + atan2(-R12, -R13)

In [ ]:
alpha, beta, gamma

In [ ]:
try:
    alpha
    beta
except:
    print('no')

In [ ]:
theta4 = gamma
theta5 = beta
theta6 = alpha

In [ ]:
R3_6_ha = R3_6.row_join(Matrix([[0],[0],[0]]))
R3_6_ha = R3_6_ha.col_join(Matrix([[0, 0, 0, 1]]))
R3_6_ha

In [ ]:
beta = atan2(-r31, sqrt(r11**2 + r21**2)) # y ,theta
gamma = atan2(r32, r33) - pi/2# x ,psi
alpha = atan2(r21, r11) - pi/2 # z ,phi

In [ ]:
s5 = sin(theta1)*r13 - cos(theta1)*r23
atan2(s5, sqrt(1-s5**2))

In [ ]:
R0_3

In [ ]:
R0_3_inv

In [ ]:
R0_3_inv[1,:] * R3_6[:,2]

In [ ]:
T0_1

In [ ]:
R0_3_inv[0,:]

In [30]:
def Q_To_E(x, y, z, w):
    ysqr = y*y
    
    t0 = +2.0 * (w * x + y*z)
    t1 = +1.0 - 2.0 * (x*x + ysqr)
    X = atan2(t0, t1)
    
    t2 = +2.0 * (w*y - z*x)
    t2 =  1 if t2 > 1 else t2
    t2 = -1 if t2 < -1 else t2
    Y = asin(t2)
    
    t3 = +2.0 * (w * z + x*y)
    t4 = +1.0 - 2.0 * (ysqr + z*z)
    Z = atan2(t3, t4)
    return X, Y, Z
ox =0.664
oy =0.11
oz =0.373
ow =0.639
Q_To_E(ox,oy,oz,ow)

(1.47012475225096, -0.362661636851847, 0.728829712582474)

In [36]:
np.random.random(3)

array([ 0.86623118,  0.79493658,  0.36197322])